# EOPatch visualizations

**Note: Most of visualizations in this notebook are dynamic and can only be seen when executing the notebook.**

First some imports. We only need basic stuff from `eolearn.core` and `geoviews` package for visualizing background layers.

In [1]:
%load_ext autoreload
%autoreload 2

import os

import geoviews as gv

from eolearn.core import EOPatch, FeatureType

We are going to load _eopatch_ from _TestEOPatch_ folder. This is example eopatch with lots of different data. 

In [2]:
data_folder = os.path.join('..', '..', 'example_data', 'TestEOPatch')
eopatch = EOPatch.load(data_folder)
eopatch

EOPatch(
  data: {
    BANDS-S2-L1C: numpy.ndarray(shape=(68, 101, 100, 13), dtype=float32)
    CLP: numpy.ndarray(shape=(68, 101, 100, 1), dtype=float32)
    NDVI: numpy.ndarray(shape=(68, 101, 100, 1), dtype=float32)
    REFERENCE_SCENES: numpy.ndarray(shape=(5, 101, 100, 13), dtype=float32)
  }
  mask: {
    CLM: numpy.ndarray(shape=(68, 101, 100, 1), dtype=uint8)
    IS_DATA: numpy.ndarray(shape=(68, 101, 100, 1), dtype=uint8)
    IS_VALID: numpy.ndarray(shape=(68, 101, 100, 1), dtype=bool)
  }
  scalar: {
    CLOUD_COVERAGE: numpy.ndarray(shape=(68, 1), dtype=float16)
  }
  label: {
    IS_CLOUDLESS: numpy.ndarray(shape=(68, 1), dtype=bool)
    RANDOM_DIGIT: numpy.ndarray(shape=(68, 2), dtype=int8)
  }
  vector: {
    CLM_VECTOR: geopandas.GeoDataFrame(columns=['TIMESTAMP', 'VALUE', 'geometry'], length=55, crs=epsg:32633)
  }
  data_timeless: {
    MAX_NDVI: numpy.ndarray(shape=(101, 100, 1), dtype=float64)
  }
  mask_timeless: {
    LULC: numpy.ndarray(shape=(101, 100, 1), dtype=

## Single plots

We can plot single element from eopatch or we can combine multiple elements either side by side or overlay them. 

### Data

Elements in _data_ are 4 dimensional Numpy Arrays. Eopatch.data['BANDS-S2-L1C'] has 4 dimension: *time, y, x* and *band*. When we do a simple _eopatch.plot_, *time* and *band* are sliders, *x and y* are x and y in the plot. With slider we can choose *time* and *band*. 

In [3]:
eopatch.plot((FeatureType.DATA, 'BANDS-S2-L1C'))

:DynamicMap   [time,BANDSS2L1C_dim]
   :Image   [x,y]   (BANDSS2L1C)

eopatch.data['NDVI'] has 4 dimension: *time, y, x* and *band*. *band* dimension has length 1, so is not shown on the plot. There is only slider for time.  

In [4]:
eopatch.plot((FeatureType.DATA, 'NDVI'))

:DynamicMap   [time]
   :Image   [x,y]   (NDVI)

### Data in RGB

Having multiple bands, one can construct RGB image. With option *rgb* you define indices of R, G and B channels, *rgb_factor* is multiplication factor for final image - default value is 3. *rgb=[3,2,1]* - for the red/R channel use 3rd channel, for green/G use 2nd channel and for blue/B use 1st channel. *rgb_factor=4* instructs to multiply result with 4 to get 'better' RGB image.

In [5]:
eopatch.plot((FeatureType.DATA, 'BANDS-S2-L1C'), rgb=[3, 2, 1], rgb_factor=4)

:HoloMap   [time]
   :DynamicMap   [band]
      :Image   [x,y]   (value)

### Data with underlying map

If there is some geographical information included in eopatch (bounding box with coordinate reference system), eopatch can be plotted on map. Undelraying map could be anything from *geoviews.tile_sources* and coordinates are automatically converted to latitute/longitude.

\* operator meand that plots are overlayed. *gv.tile_source* is allways in the background. 

In [6]:
eopatch.plot((FeatureType.DATA, 'NDVI')) * gv.tile_sources.Wikipedia

:DynamicMap   [time]
   :Overlay
      .Image.I :Image   [x,y]   (NDVI)
      .WMTS.I  :WMTS   [Longitude,Latitude]

## Plot tools

On the right of the plot there are some tools. They could be different available based on the plot shown.  

* Pan: move the plot (clik and drag)
* Box zoom: zoom chosen area (click and drag) 
* Wheel zoom: scrolling the wheel zooms
    * in the plot area zooms everything
    * scrollling the wheel over axes zooms only the axes (does not work on geographic plots)
* Reset: reset to original size
* Hover: show info about data under cursor.
* Save: save plot as .png file.

**Combining multiple tools**:

* \* overlay multiple plots
* \+ multiple plots side by side

### Data * mask

Here *mask* is in front of *data*. First plot is in the background, additional plots are placed in front.  *mask* is *boolean* or *int*. Values 1 or *True* are colored white, 0 or *False* are transparent. How transparent is 1/True can be set by *.opts(alpha=0.3)* paramether. That way we can see what is behind the mask.  

Wiht upper slider one can change *BANDS-S2-L1C* channel, this does not effect *CLM*. Lower slider is for choosing the date, this effects both *BANDS-S2-L1C* and *CLM*. 

In [7]:
eopatch.plot((FeatureType.DATA, 'BANDS-S2-L1C')) * eopatch.plot((FeatureType.MASK, 'CLM')).opts(alpha=0.3)

:DynamicMap   [time,BANDSS2L1C_dim]
   :Overlay
      .Image.I  :Image   [x,y]   (BANDSS2L1C)
      .Image.II :Image   [x,y]   (CLM)

### Data + mask 

With *+* operator, we can put multiple plots side by side. *opt(bgcolor='color')* changes background color of plot. 

In [8]:
eopatch.plot((FeatureType.DATA, 'BANDS-S2-L1C')) + eopatch.plot((FeatureType.MASK, 'CLM')).opts(bgcolor='blue')

:Layout
   .DynamicMap.I  :DynamicMap   [time,BANDSS2L1C_dim]
      :Image   [x,y]   (BANDSS2L1C)
   .DynamicMap.II :DynamicMap   [time]
      :Image   [x,y]   (CLM)

## Line plots

*Scalar* and *lable* have only 1 or 2 dimensions, so they are plot like line. 

In [9]:
eopatch.plot((FeatureType.SCALAR, 'CLOUD_COVERAGE'))

:Curve   [time]   (CLOUD_COVERAGE)

## Vector plots

Data in eopatch can be also in vector form, that is geopandas GeoDataFrame. Here it is ploted on image background (*gv.title_sources.* is always in the background). Vector plots could take longer time to produce. If there is time dimension, empty times are added to visualization (when no data is present).

*timestamp* column (name of the GeoDataFrame column with dates) and *geometry* column (name of the GeoDataFrame column with geometry data) could be specified as parameters of plot function. 

In [10]:
eopatch.plot((FeatureType.VECTOR, 'CLM_VECTOR')) * gv.tile_sources.EsriImagery

:HoloMap   [time]
   :Overlay
      .Polygons.I :Polygons   [Longitude,Latitude]
      .WMTS.I     :WMTS   [Longitude,Latitude]

Ploting *vector_timeless* data, one can speciy *value dimensions (vdims)* - data is colored based on that column values (must be numerical).  

In [11]:
eopatch.plot((FeatureType.VECTOR_TIMELESS, 'LULC'), vdims=['RABA_ID'])

:Polygons   [Longitude,Latitude]   (RABA_ID)

## Options

Size of the plot. 

In [12]:
eopatch.plot((FeatureType.DATA, 'NDVI')).opts(height=400, width=600)

:DynamicMap   [time]
   :Image   [x,y]   (NDVI)

We can change title of the plot. 

In [13]:
eopatch.plot((FeatureType.SCALAR, 'CLOUD_COVERAGE')).opts(title="CLOUD COVERAGE")

:Curve   [time]   (CLOUD_COVERAGE)

Sizing of the font in plot is done with *fontsize* option.  

In [14]:
eopatch.plot((FeatureType.SCALAR, 'CLOUD_COVERAGE')).opts(fontsize={'title': 16, 'labels': 14, 
                                                                    'xticks': 6, 'yticks': 12},
                                                         title='Cloud coverage',
                                                         width=600, height= 400)

:Curve   [time]   (CLOUD_COVERAGE)

Position of toolbar could be changed and tools could be added (in this example *hoover*). 

In [15]:
eopatch.plot((FeatureType.VECTOR_TIMELESS, 'LULC'), vdims=['RABA_ID']).opts(toolbar='above', tools=['hover'])

:Polygons   [Longitude,Latitude]   (RABA_ID)

There are lots of options for axes. We can name them.

In [16]:
eopatch.plot((FeatureType.DATA, 'NDVI')).opts(xlabel='longitude', ylabel='latitude')

:DynamicMap   [time]
   :Image   [x,y]   (NDVI)

Or we can remove them. 

In [17]:
eopatch.plot((FeatureType.DATA, 'NDVI')).opts(xaxis=None, yaxis=None)

:DynamicMap   [time]
   :Image   [x,y]   (NDVI)

Colormap can be adjusted. 

In [18]:
eopatch.plot((FeatureType.DATA, 'NDVI')).opts(cmap='YlGn')

:DynamicMap   [time]
   :Image   [x,y]   (NDVI)